In [40]:
import pandas as pd
import numpy as np
import scipy.io as sio

D = sio.loadmat('data.mat')
#dict_keys(['__header__', '__version__', '__globals__', 'Censored', 'Integ_Symb_Types', 'Integ_Symbs', 'Integ_X', 'Integ_X_raw', 'Patients', 'Subtypes', 'Survival'])


#X = np.asarray([x[0] for x in D['Integ_X']]).astype('float32')
T = np.asarray([t[0] for t in D['Survival']]).astype('float32')
O = 1 - np.asarray([c[0] for c in D['Censored']]).astype('int32')
#del(df)
df = pd.DataFrame(data=T, columns=["Survival"])
#df=pd.DataFrame
df['Censored'] = O
#df['Integ_X'] = X
df.reset_index()
df.head()


,Survival,Censored
0,144.0,1
1,393.0,1
2,470.0,0
3,211.0,1
4,691.0,1
